In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

#  Introduction
This EDA is taking these 4 steps.  
Step1 : Overview the data  
Step2 : See the relationship between...   
 <span>　</span>a) "AMT_INCOME_TOTAL" & repayment or non-repayment rate   
 <span>　</span>b) "OCCUPATION_TYPE" & repayment or non-repayment rate  
Step3 : Picking up other features that will help to predict custmer's repayment ability  
Step4 : Plot non-repayment rate for all selected features
  
In Step1, overview the data to check what kind of data in there and if there is any missing values. Then, define non-repayment rate from target data.  
In Step2, I forcus on "AMT_INCOME_TOATL" & "OCCUPATION_TYPE" features because, generaly thinking, these features seem to relate to custmer's repayment ability. It's easy to imagine that low income cause of default so I expect the tendecy which is "if income gets lower then non-repayment rate becomes higher and if income gets higher then non-repayment rate becomes lower". I will test this "theory" in step2. Then I will check "OCCUPATION_TYPE"  because I believe it relates to income level and also occupation infomation can be reflected personal feature more than income.   
So, I will check if "AMT_INCOME_TOATL" & "OCCUPATION_TYPE"  can help to predict custmer's repayment ability or not in Step2.   
In Step3,  I will pick up other important features with using random forest analysis.  
In Step4, Plot non-repayment rate for all selected fesatures  

# Step1.  Overview the data
overview the data to check what kind of data in there and  if there is any missing values. Then define non-repayment rate from target data.

>### Load the data & take a look what kind of data in there

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_application_train = pd.read_csv("../input/application_train.csv")
pd.options.display.max_columns = len(data_application_train.columns)
data_application_train.head()

> ### Check missing data
Plot missing data rate with "missingno" module and calculate missing rate for "AMT_INCOME_TOATL" & "OCCUPATION_TYPE"

In [ ]:
import missingno as msno
msno.bar(data_application_train)

In [ ]:
total = len(data_application_train)
sum_income = data_application_train["AMT_INCOME_TOTAL"].isnull().sum()
sum_occupation = data_application_train["OCCUPATION_TYPE"].isnull().sum()

print("AMT_INCOME_TOTAL  num of missing data={}　 missing data rate={:.2f}[%]".format(sum_income, 100*sum_income/total))
print("OCCUPATION_TYPE   num of missing data={}　 missing data rate={:.2f}[%]".format(sum_occupation, 100*sum_occupation/total))

>### Define non-repayment rate from target data
"Home_credit_columns_description.csv" says target data is defined as    
*"1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases"*  
Here, I intoroduce "non-repayment rate" and define it as  $\frac {sum(1)}  {(sum(1) + sum (0))}$ .   If I calculate "non-repayment rate" across the whole data, then result is 8:07[%] ( It also can be calculated by mean of data because target data is 0 & 1 )

In [ ]:
sns.countplot(x="TARGET", data=data_application_train)
plt.title("Data volume in each class('Target')")
plt.show()

count_1 = (data_application_train["TARGET"] == 1).sum()
count_0 = (data_application_train["TARGET"] == 0).sum()
print("Non-repayment rate={:.2f}[%]".format((count_1/(count_0+count_1))*100))

# Step2.  See the relationship between...   
 **<span>　</span>a) "AMT_INCOME_TOTAL" & Non-repayment rate**  
 **<span>　</span>b) "OCCUPATION_TYPE" & Non-repayment rate**

###  See the relationship between "AMT_INCOME_TOTAL" and  Non-repayment rate
Test the theory "low income = high non-repayment rate" & "high income = low non-repayment rate".  
**Method**
1. Benning income data
2. Calculate Non-repayment rate in each the bins
3. Plot "Binned income" vs "Non-repayment"

In [ ]:
#Extract require data
data_to_use = data_application_train[["TARGET","AMT_INCOME_TOTAL","OCCUPATION_TYPE"]]

To decide bin width, checking "AMT_INCOME_TOTAL" distribution below.   
From the distribution, we understand we can't use max & min value to decide bin width becasue there are outliers in the data. So I'm going to use quartile value to decide bin width.

In [ ]:
sns.violinplot(y='AMT_INCOME_TOTAL', data=data_to_use)
plt.ylabel("Income")
plt.title("Income distribution")
plt.show()
data_to_use.describe()

Now binning data by quartile value

In [ ]:
def binning_data(data_source, col_name, binned_col_name, num_of_bin=10):
    """
    Binning the specified column data
    & add the new column that has bin label to original data
    
    parameter
    --------------
    data_source : Pandas dataframe
    col_name : string 
    binned_col_name : string 
    num_of_bin : int
    
    return
    --------------
    data_source : pandas dataframe
    
    """
    #Check quartile value
    data_info = data_source.describe()
    bin_min = data_info.loc["25%", col_name]
    bin_max = data_info.loc["75%", col_name]
    IQR = bin_max - bin_min
    #Use interquartile value to decide bin_min & max value
    bin_min = bin_min-IQR*1.5 if bin_min > IQR*1.5 else 0
    bin_max += IQR*1.5
    
    bin_width = (int)((bin_max - bin_min) / num_of_bin)
    #bins = [value for value in gen_range(bin_min, bin_max, bin_width )]
    bins = [value for value in range((int)(bin_min), (int)(bin_max), bin_width )]
    labels = [i for i in range(0, len(bins)-1)]
    binned_label = pd.cut(data_source[col_name], bins=bins, labels=labels)
    data_source[binned_col_name] = binned_label
    
    for i in range(0, len(bins)-1):
        print("range label={} : range={:.5f}~{:.5f}".format(i, bins[i], bins[i+1]) )
    
    return data_source

In [ ]:
#Binning data
data_to_use = binning_data(data_to_use, "AMT_INCOME_TOTAL", "BINNED_AMT_INCOME_TOTAL", 8)

1. Income distribution after benning

In [ ]:
#Check distribution
income_destribution_w_income_range = data_to_use.groupby("BINNED_AMT_INCOME_TOTAL", as_index=False).count()
plt.bar(income_destribution_w_income_range["BINNED_AMT_INCOME_TOTAL"]
        , income_destribution_w_income_range["TARGET"])
plt.xlabel('Low<- Income range label ->High)')
plt.ylabel("count")
plt.title("Income distribution")
for x, y in zip(income_destribution_w_income_range["BINNED_AMT_INCOME_TOTAL"]
                , income_destribution_w_income_range["TARGET"]):
    plt.text(x, y, y, ha='center', va='bottom')
plt.show()

Finally, calculate non-repayment rate in each bins & plot them

In [ ]:
#Cal non-repayment rate
non_repayment_rate_w_income_range = data_to_use.groupby("BINNED_AMT_INCOME_TOTAL", as_index=False).mean()
non_repayment_rate_w_income_range["TARGET"] *= 100 #% notation

#Bar plot
plt.figure(figsize=(10,6))
plt.bar(non_repayment_rate_w_income_range["BINNED_AMT_INCOME_TOTAL"]
        , non_repayment_rate_w_income_range["TARGET"], color="Blue")
plt.xlabel('Low<- Income range label ->High')
plt.ylabel('non-payment rate[%]')
plt.ylim(0,10)
plt.title("non-payment rate")

for x, y in zip(non_repayment_rate_w_income_range["BINNED_AMT_INCOME_TOTAL"]
                , non_repayment_rate_w_income_range["TARGET"]):
    plt.text( x, y, str("{:.2f}").format(y), ha='center', va='bottom')
plt.show()

####  Conclusion of  "AMT_INCOME_TOTAL" vs  "Non-repayment rate" analysis
1. What I expected is "low income => high non-repayment rate" . However no significant differernce is observed among Range0 to 4 (Across low income to middle).  
2. Highest non-repayment rate is observed in Range2, which is considered average income range.
3. "high income => low non-repayment rate" seems right. Range6 & 7 show lower non-repayment rate than other
4. Non-repayment rate difference between higest & lowest is 2.4%. It is not the big difference as I expected.    
5. From this analysis,  I can say...  
"Low income isn't the convincing reason of loan rejection" or "Home credit may have careful loan assessment for low income customers (this is why we don't see significant non-repayment rate differernce in the result)"  
6. However, becasue no significant non-repayment rate difference is observed among the income ranges,  I can't conclude that "AMT_INCOME_TOTAL" is important feature. I will test  "AMT_INCOME_TOTAL" importance with Random forest analysis in Step3

###  See the relationship between "OCCUPATION_TYPE" and  Non-repayment rate
**Method**
1. Calculate non-repayment rate in each "OCCUPATION_TYPE"
2. Plot "OCCUPATION_TYPE vs "Non-repayment"

In [ ]:
#Cal non-repayment rate for each occupation type
non_repayment_rate_w_occupation = data_to_use.groupby("OCCUPATION_TYPE", as_index=False).mean()
non_repayment_rate_w_occupation["TARGET"] *= 100
#non_repayment_rate_w_occupation["OCCUPATION_TYPE_LABEL"] = [i for i in range(0, len(non_repayment_rate_w_occupation))]
print(non_repayment_rate_w_occupation)

#Bar plot
plt.figure(figsize=(30,12))
plt.bar(non_repayment_rate_w_occupation.index, non_repayment_rate_w_occupation["TARGET"], color="Blue")
plt.xlabel("OCCUPATION_TYPE")
plt.ylabel("Non-payment rate[%]")
plt.title("Non-payment rate")

for x, y in zip(non_repayment_rate_w_occupation.index, non_repayment_rate_w_occupation["TARGET"]):
    plt.text(x, y, str("{:.2f}").format(y), ha='center', va='bottom')

plt.xticks(non_repayment_rate_w_occupation.index, non_repayment_rate_w_occupation["OCCUPATION_TYPE"])
plt.show()

####  Conclusion of  "OCCUPATION_TYPE" vs  "Non-repayment rate" analysis
1. High non-repayment rate is observed in Labors. Espacilly low-skill labors marks highest rate 17.15%.   
2. Non-repayment rate tends to high in the job that doesn't require high-skill, high-education like waiter, drivers, security staff etc...  Education history may also be a important feature. 
3. Accountant has Lowest non-repayment rate 4.53%. I suppose they are good at finacial planning & control because they are accountant.   
4. As a result, we see siginificant non-repayment rate difference among occupation type so I conclude that ”OCCUPATION_TYPE” can be a important feature.

###  Conclusion of  Step2
1. "AMT_INCOME_TOTAL" will be tested with random forest analysis in Step3 to find out if it's important feature or not  
2. ”OCCUPATION_TYPE” can be a important feature  
3. Generally thinking, income relate to occupation type. However non-repayment rate difference is observed well among occupation type rather than among income range.  Thus the features that describe customer's social role, resposibility and surrounding environment seems to be more important feature than income. 

# Step3.  Picking up other features that will help to predict custmer's repayment ability    

**method**
1. Drop the feature that have much missing data
2. Encoding object tyep data
3. Calculate importance with Random forest analysis

1. Drop the feature that have much missing data  
If missing data rate is over 40%, then drop those features from this analysis.   (I will run same analysis without dropping missing data later and compare the result. )

In [ ]:
#1.Drop high missing value data
#Set drop criteria as 40%. If missing value rate is over 40%, then drop those features.
REDUCTION_CRITERIA_FOR_MISSING_DATA = 40 #[%] 
total = len(data_application_train)
col_name = data_application_train.columns
missing_data_rate = [100*data_application_train[col_name[i]].isnull().sum() / total for i in range(0, len(col_name))]

for i in range(0, len(col_name)):
    if REDUCTION_CRITERIA_FOR_MISSING_DATA < missing_data_rate[i]:
        del data_application_train[col_name[i]]

msno.bar(data_application_train)
print("Num of Featurs: Before reduction {} => After Reduction {}".format(len(col_name), len(data_application_train.columns.values)))

Encoding object tyep data  
All object type data are categorical variable (nominal scale) then replace those value, include "nan" , to any number to run Random forest analysis in next.

In [ ]:
object_data_application_train = data_application_train.select_dtypes(['object'])
object_data_application_train.head()

In [ ]:
#Replace object type data to number
object_col_name = object_data_application_train.columns.values
array_object_to_int = np.array([object_data_application_train[object_col_name[i]].unique() for i in range(0,len(object_col_name))])

for i in range(0,len(object_col_name)):
    labels, uniques = pd.factorize(data_application_train[object_col_name[i]])
    data_application_train[object_col_name[i]] = labels   

#Repalce "NAN" to -1    
data_application_train = data_application_train.replace(np.nan, -1)

Run Random forest analysis

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()

In [ ]:
#Drop SK_ID_CURR
del data_application_train["SK_ID_CURR"]
#Drop OCCUPATION_TYPE (Already done in Step2)
del data_application_train["OCCUPATION_TYPE"]

In [ ]:
target = data_application_train['TARGET']
feature = data_application_train.iloc[:,1:len(data_application_train.columns)]
model.fit(feature, target)

In [ ]:
rank = np.argsort(-model.feature_importances_)
f, ax = plt.subplots(figsize=(11, 11)) 
sns.barplot(x=model.feature_importances_[rank], y=data_application_train.columns.values[rank], orient='h')
ax.set_xlabel("Importance")
plt.tight_layout()
plt.show()

In [ ]:
#Pick up top10 important feature
for i in range(0, 10):
    print("Rank{} => {} (Importance={:.3f})".format(i+1, 
                                                    data_application_train.columns.values[rank[i]], 
                                                    model.feature_importances_[rank[i]]))

Run Random forest analysis without dropping missing values

In [ ]:
train_data =  pd.read_csv("../input/application_train.csv")

In [ ]:
#Replace object type data to number
object_train_data = train_data.select_dtypes(['object'])
object_col_name = object_train_data.columns.values
array_object_to_int = np.array([object_train_data[object_col_name[i]].unique() for i in range(0,len(object_col_name))])

for i in range(0,len(object_col_name)):
    labels, uniques = pd.factorize(train_data[object_col_name[i]])
    train_data[object_col_name[i]] = labels   

#Repalce "NAN" to -1    
train_data = train_data.replace(np.nan, -1)

In [ ]:
#Drop SK_ID_CURR
del train_data["SK_ID_CURR"]
#Drop OCCUPATION_TYPE (Already done in Step2)
del train_data["OCCUPATION_TYPE"]

In [ ]:
target = train_data['TARGET']
feature = train_data.iloc[:,1:len(train_data.columns)]
model.fit(feature, target)

In [ ]:
rank = np.argsort(-model.feature_importances_)
f, ax = plt.subplots(figsize=(15, 15)) 
sns.barplot(x=model.feature_importances_[rank], y=train_data.columns.values[rank], orient='h')
ax.set_xlabel("Importance")
plt.tight_layout()
plt.show()

In [ ]:
for i in range(0, 10):
    print("Rank{} => {} (Importance={:.3f})".format(i+1, train_data.columns.values[rank[i]], model.feature_importances_[rank[i]]))

Comparing Random forest analysis result w/ dropping missing values case and w/o dropping missing value case

| Ranking | w/o missing value | w/ missing value |
|:-----------:|:-----------:|:------------:|
| 1 | **ORGANIZATION_TYPE (Importance=0.077)**  | **EXT_SOURCE_1 (Importance=0.058)**  |
| 2 | EXT_SOURCE_2 (Importance=0.058)  | EXT_SOURCE_2 (Importance=0.044) |
| 3 | REGION_POPULATION_RELATIVE (Importance=0.058)  | DAYS_EMPLOYED (Importance=0.039)  |
| 4 | DAYS_REGISTRATION (Importance=0.056)  | DAYS_REGISTRATION (Importance=0.039) |
| 5 | DAYS_EMPLOYED (Importance=0.055) | REGION_POPULATION_RELATIVE (Importance=0.039) |
| 6 | AMT_CREDIT (Importance=0.051)  | DAYS_BIRTH (Importance=0.035) |
| 7 | DEF_60_CNT_SOCIAL_CIRCLE (Importance=0.050) | AMT_CREDIT (Importance=0.034) |
| 8 | DAYS_BIRTH (Importance=0.049) | DEF_60_CNT_SOCIAL_CIRCLE (Importance=0.033) |
| 9 | AMT_INCOME_TOTAL (Importance=0.046) | AMT_INCOME_TOTAL (Importance=0.032) |
| 10 | **CNT_CHILDREN (Importance=0.042)** | **AMT_ANNUITY (Importance=0.029)** |

Importance is changed however 8 common features are in the both top10 rank.  "ORGANIZATION_TYPE" and "EXT_SOURCE_1" are switched between w/ dropping and w/o dropping.  Also "CNT_CHILDREN" and "AMT_AN NUITY" are switched.  I dropped "EXT_SOURCE_1" and  "AMT_AN NUITY" in first Random forest analysis becasue they have over 40% missing value. However, from 2nd Random forest analysis, it turns out that actually they are important features even they have missing values.   

### Conclusion of Step3
As a result, 12 festures below are picked up.

| No| Selected feature | 
|:-----------:|:-----------:|
| 1 | ORGANIZATION_TYPE  |
| 2 | EXT_SOURCE_1  |
| 3 | EXT_SOURCE_2 |
| 4 | REGION_POPULATION_RELATIVE |
| 5 | DAYS_REGISTRATION  |
| 6 | DAYS_EMPLOYED |
| 7 | AMT_CREDIT |
| 8 | DEF_60_CNT_SOCIAL_CIRCLE  |
| 9 | DAYS_BIRTH  |
| 10 | AMT_INCOME_TOTAL |
| 11 | CNT_CHILDREN |
| 12 |AMT_ANNUITY |

# Plot non-repayment rate for all selected features  
As a result, some plots show strong relationship between selected feature and non-repayment rate. For example "Organization type". We can see significant non-repayment differece among parameters as same as "occupation type". So it seems customer's surronding environment is a key to pridect the repayment ability.   
And also "EXT_SOURCE_1 & 2". I need to study about these features more. However these show the strong trend which lower value of "EXT_SOURCE_1 & 2" shows higher non-repayment rate and higher value of "EXT_SOURCE_1 & 2" shows lower non-repayment rate. 

In [ ]:
data =  pd.read_csv("../input/application_train.csv")

In [ ]:
def plot_non_repayment_rate(data_source, col_name, flag_rename_x_label=False):
    """
    Plot non-repayment rate. (Xaxis=Col_name, Yaxis=non-repayment rate)
    
    parameter
    --------------
    data_source : Pandas dataframe
    col_name : string 
    flag_rename_x_label : bool
    
    return
    --------------
    None
    """
    
    #Cal non-repayment rate
    non_repayment_rate = data_source.groupby(col_name, as_index=False).mean()
    non_repayment_rate["TARGET"] *= 100
    #print(non_repayment_rate)
    
    #Bar plot
    plt.figure(figsize=(25,10))
    plt.bar(non_repayment_rate.index, non_repayment_rate["TARGET"], color="Blue")
    plt.xlabel(col_name, fontsize=18)
    plt.ylabel("Non-payment rate[%]", fontsize=18)
    plt.title("Non-payment rate")
    plt.tight_layout()
    
    for x, y in zip(non_repayment_rate.index, non_repayment_rate["TARGET"]):
        plt.text(x, y, str("{:.2f}").format(y), ha='center', va='bottom')
        if flag_rename_x_label == True:
            plt.xticks(non_repayment_rate.index, non_repayment_rate[col_name])
    plt.show()

In [ ]:
def gen_range(value_start, value_end, value_step):
    """
    Extend range() so it can handle float type data
    ----------------
    Parameter
        value_start:float
        value_end:float
        value_step:float
    ----------------
    ----------------
    Return
    　　value:float
    ----------------
    """
    value = value_start
    while value+value_step < value_end:
     yield value
     value += value_step

In [ ]:
def binning_data2(data_source, col_name, binned_col_name, num_of_bin=10):
    """
    parameter
    --------------
    data_source : Pandas dataframe
    col_name : string 
    binned_col_name : string 
    num_of_bin : int
    
    return
    --------------
    data_source : pandas dataframe
    
    """
    #Use interquartile value to decide bin_min & max value
    data_info = data_source.describe()
    bin_min = data_info.loc["25%", col_name]
    bin_max = data_info.loc["75%", col_name]
    IQR = bin_max - bin_min
    bin_min = bin_min-IQR*1.5 if bin_min > IQR*1.5 else 0
    bin_max += IQR*1.5
    
    bin_width = (bin_max - bin_min) / num_of_bin
    bins = [value for value in gen_range(bin_min, bin_max, bin_width )]
    labels = [i for i in range(0, len(bins)-1)]
    binned_label = pd.cut(data_source[col_name], bins=bins, labels=labels)
    data_source[binned_col_name] = binned_label

    for i in range(0, len(bins)-1):
        print("range label={} : range={:.5f}~{:.5f}".format(i, bins[i], bins[i+1]) )
    
    return data_source

In [ ]:
#ORGANIZATION_TYPE
plot_non_repayment_rate(data, 'ORGANIZATION_TYPE', False)
tmp = data["ORGANIZATION_TYPE"].unique()
for i in range(0, len(tmp)):
    print("{}:{}".format(i, tmp[i]))

In [ ]:
#EXT_SOURCE_1
plot_non_repayment_rate(binning_data2(data, 'EXT_SOURCE_1', 'BIN_EXT_SOURCE_1', 8), 'BIN_EXT_SOURCE_1')

In [ ]:
#EXT_SOURCE_2
plot_non_repayment_rate(binning_data2(data, 'EXT_SOURCE_2', 'BIN_EXT_SOURCE_2', 8), 'BIN_EXT_SOURCE_2')

In [ ]:
#REGION_POPULATION_RELATIVE
plot_non_repayment_rate(binning_data2(data, 'REGION_POPULATION_RELATIVE', 'BIN_REGION_POPULATION_RELATIVE', 8), 'BIN_REGION_POPULATION_RELATIVE')

In [ ]:
#AMT_CREDIT
plot_non_repayment_rate(binning_data(data, 'AMT_CREDIT', 'BIN_AMT_CREDIT', 8), 'BIN_AMT_CREDIT')

In [ ]:
#DEF_60_CNT_SOCIAL_CIRCLE
plot_non_repayment_rate(data, 'DEF_60_CNT_SOCIAL_CIRCLE', True)

In [ ]:
#CNT_CHILDREN
plot_non_repayment_rate(data, 'CNT_CHILDREN', True)

In [ ]:
#AMT_ANNUITY
plot_non_repayment_rate(binning_data2(data, 'AMT_ANNUITY', 'BIN_AMT_ANNUITY', 8), 'BIN_AMT_ANNUITY')